In [1]:
import sys
from pathlib import Path
import os 
import pandas as pd
ROOT = Path().resolve().parent.parent
sys.path.append(str(ROOT))

In [6]:
from src.preprocess.image_quality_metrics import ImageQualityMetrics

processed_folder = ROOT / 'data/processed'
image_folders = os.listdir(processed_folder)
image_folders.sort()

df = pd.DataFrame()
rows = []

for folder in image_folders:
    if folder == '.DS_Store':
        continue

    reference_path = processed_folder / folder / 'groundtruth.fits'

    sim_folders = [f for f in os.listdir(processed_folder / folder) if 'sim' in f]

    for sim_folder in sim_folders:
        sim_images = [f for f in os.listdir(processed_folder / folder / sim_folder) if 'tclean' in f]
        for rec in sim_images:
            rec_folder = processed_folder / folder / sim_folder / rec

            image_path    = rec_folder / f"{rec}.image"
            residual_path = rec_folder / f"{rec}.residual"

            iqm = ImageQualityMetrics(
                image_path=str(image_path), 
                residual_path=str(residual_path), 
                reference_path=str(reference_path),
            )
            snr = iqm.snr()
            ssim = iqm.ssim()
            psnr_no_reference = iqm.psnr_no_reference()
            psnr_reference = iqm.psnr_reference()
            peak = iqm.peak()
            rms = iqm.rms()
            
            # Additional metrics
            sharpness = iqm.sharpness()
            median = iqm.median()
            entropy = iqm.entropy()
            noise = iqm.noise()


            data_map = {
                'path': str(image_path.relative_to(ROOT)),
                'object': folder,
                'sim': sim_folder,
                'reconstruction': rec,
                'snr': snr,
                'ssim': ssim,
                'psnr_no_reference': psnr_no_reference,
                'psnr_reference': psnr_reference,
                'peak': peak,
                'rms': rms,
                'sharpness': sharpness,
                'median': median,
                'entropy': entropy,
                'noise': noise
            }
            rows.append(data_map)


df = pd.DataFrame(rows)
df.to_csv(ROOT / 'data' / 'image_quality_metrics.csv', index=False)

In [7]:
df = pd.read_csv(ROOT / 'data' / 'image_quality_metrics.csv')
df

,path,object,sim,reconstruction,snr,ssim,psnr_no_reference,psnr_reference,peak,rms,sharpness,median,entropy,noise
0,data/processed/img_0000/sim2/tclean_multiscale...,img_0000,sim2,tclean_multiscale_2500,7.978179,0.382072,1.765622,23.657766,255.0,144.425051,13.144426,0.067850,15.996876,0.436437
1,data/processed/img_0000/sim2/tclean_hogbom_400...,img_0000,sim2,tclean_hogbom_40000,9.336039,0.379462,1.431027,24.745150,255.0,178.193650,16.350446,0.163347,15.995575,0.677921
2,data/processed/img_0000/sim2/tclean_mem_1000/t...,img_0000,sim2,tclean_mem_1000,7.377021,0.370901,1.366185,23.255635,255.0,186.651122,11.656647,0.056739,15.995819,0.340941
3,data/processed/img_0000/sim1/tclean_multiscale...,img_0000,sim1,tclean_multiscale_2500,3.915633,0.457655,1.617618,11.070400,255.0,157.639236,34.894677,1.914712,15.993732,3.230500
4,data/processed/img_0000/sim1/tclean_hogbom_400...,img_0000,sim1,tclean_hogbom_40000,4.868769,0.489562,1.483328,13.613603,255.0,171.910701,31.005688,1.801010,15.994758,2.875321
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1707,data/processed/img_0283/sim2/tclean_hogbom_400...,img_0283,sim2,tclean_hogbom_40000,16.121808,0.591912,1.966662,28.479432,255.0,129.661297,14.828530,0.116872,15.997589,0.524034
1708,data/processed/img_0283/sim2/tclean_mem_1000/t...,img_0283,sim2,tclean_mem_1000,16.663782,0.643644,1.915417,28.876434,255.0,133.130306,13.341324,0.043548,15.997589,0.310159
1709,data/processed/img_0283/sim1/tclean_multiscale...,img_0283,sim1,tclean_multiscale_2500,5.308036,0.349318,1.625265,13.827060,255.0,156.897536,18.772305,0.666102,15.996826,1.224755
1710,data/processed/img_0283/sim1/tclean_hogbom_400...,img_0283,sim1,tclean_hogbom_40000,5.658521,0.370528,1.431019,14.862466,255.0,178.194747,19.915585,1.727112,15.997040,2.696233
